<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BOB_MVP_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## libraries

In [ ]:
!pip install flask -q
!pip install flask_migrate -q
!pip install flask_sqlalchemy -q
!pip install anthropic -q
!pip install colab-env --quiet
!pip install flask_cors -q


# --- Installation ---
#flask_sqlalchemy
!pip install flask_ngrok -q
!pip install pyngrok -q  # Install pyngrok directly
!pip install --upgrade flask_ngrok pyngrok -q
!pip install flask_login -q